In [1]:

from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.queryTranslator import QueryType, QueryTranslator
from alhazen.utils.searchEngineUtils import EuroPMCQuery, ESearchQuery
from alhazen.utils.local_literature_db import LocalLiteratureDb, QuerySpec
from alhazen.schema_sqla import ScientificPublicationCollection as SciPubColl, ScientificPublication as SciPub

from sqlalchemy import inspect

import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import datetime
import json


In [2]:
# Variables that describe an airtable database. 
at_key = 'keyJh5ZAdUJzHrMbM'
at_file = 'appHwopxPL0phm3c2'

# Updated instructions allow us to map different input sheets into a standard struture and then run queries over EPMC
at_sheets = [
    ('Nature Methods', 'ID', 'TERMS', {}, ['JOURNAL'])
]

# Variables that describe the databricks configuration
catalog = 'gburns'
database = 'nature_methods' 
pubmed_api_key = 'd086451c882fabace54d7b049b6fb8481908'
loc = '/Users/gburns/alhazen/'

In [3]:
atu = AirtableUtils(at_key)

qs = QuerySpec('Imaging Journals', 'ID', 'TERMS', 'CORPUS_NAME', {}, ['JOURNAL'])
cdf = atu.read_airtable(at_file, qs.name)
cdf = qs.process_cdf(cdf)
qt = QueryTranslator(cdf.sort_values('ID'), 'ID', 'QUERY', 'CORPUS_NAME')

qs2 = QuerySpec('Imaging', 'ID', 'TERMS', 'CORPUS_NAME', {}, ['METHODS', 'TITLE', 'ABSTRACT'])
sqdf = atu.read_airtable(at_file, qs2.name)
sqdf = qs2.process_cdf(sqdf)
qt2 = QueryTranslator(sqdf.sort_values('ID'), 'ID', 'QUERY', 'CORPUS_NAME')

.|


100%|██████████| 2/2 [00:00<00:00, 2081.02it/s]

.

|


100%|██████████| 1/1 [00:00<00:00, 2078.45it/s]


In [5]:
lldb = LocalLiteratureDb(loc, 'imagetech_landscape')

In [5]:
lldb.add_corpus_from_epmc(qt, qt2, sections=qs.sections, sections2=qs2.sections)

100%|██████████| 1/1 [00:00<00:00, 3775.25it/s]


https://www.ebi.ac.uk/europepmc/webservices/rest/search?format=JSON&pageSize=1000&synonym=TRUE&resultType=core&query=((JOURNAL:"Nature methods")) AND (((METHODS:"imaging" OR TITLE:"imaging" OR ABSTRACT:"imaging") OR (METHODS:"image" OR TITLE:"image" OR ABSTRACT:"image") OR (METHODS:"opto" OR TITLE:"opto" OR ABSTRACT:"opto") OR (METHODS:"microscopy" OR TITLE:"microscopy" OR ABSTRACT:"microscopy") OR (METHODS:"microsope" OR TITLE:"microsope" OR ABSTRACT:"microsope") OR (METHODS:"spectroscopy" OR TITLE:"spectroscopy" OR ABSTRACT:"spectroscopy") OR (METHODS:"radiology" OR TITLE:"radiology" OR ABSTRACT:"radiology") OR (METHODS:"radiological" OR TITLE:"radiological" OR ABSTRACT:"radiological"))), 1373 European PMC PAPERS FOUND


100%|██████████| 2/2 [00:13<00:00,  6.65s/it]


 Returning 1373


100%|██████████| 1373/1373 [00:00<00:00, 2065.78it/s]


https://www.ebi.ac.uk/europepmc/webservices/rest/search?format=JSON&pageSize=1000&synonym=TRUE&resultType=core&query=((JOURNAL:"bioRxiv : the preprint server for biology")) AND (((METHODS:"imaging" OR TITLE:"imaging" OR ABSTRACT:"imaging") OR (METHODS:"image" OR TITLE:"image" OR ABSTRACT:"image") OR (METHODS:"opto" OR TITLE:"opto" OR ABSTRACT:"opto") OR (METHODS:"microscopy" OR TITLE:"microscopy" OR ABSTRACT:"microscopy") OR (METHODS:"microsope" OR TITLE:"microsope" OR ABSTRACT:"microsope") OR (METHODS:"spectroscopy" OR TITLE:"spectroscopy" OR ABSTRACT:"spectroscopy") OR (METHODS:"radiology" OR TITLE:"radiology" OR ABSTRACT:"radiology") OR (METHODS:"radiological" OR TITLE:"radiological" OR ABSTRACT:"radiological"))), 4552 European PMC PAPERS FOUND


100%|██████████| 5/5 [00:42<00:00,  8.44s/it]


 Returning 4552


100%|██████████| 4552/4552 [00:08<00:00, 562.51it/s] 


In [6]:
cs = lldb.session.query(SciPubColl).all()
for c in cs:
    print(c)

AttributeError: 'NoneType' object has no attribute 'query'

In [7]:
import alhazen.schema_sqla as linkml_sqla
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker

if lldb.session is None:
    session_class = sessionmaker(bind=lldb.engine)
    lldb.session = session_class()

cp = lldb.session.query(SciPubColl) \
            .join(SciPubColl.has_part) \
            .filter(SciPubColl.id=='0.0').first()
print(len(cp.has_part))

1295


In [8]:
q = lldb.session.query(SciPubColl) \
            .join(SciPubColl.has_part)

In [9]:
q = lldb.session.query(SciPubColl) \
            .join(SciPubColl.has_part)

#inst = inspect(SciPub)
#attr_names = [c_attr.key for c_attr in inst.mapper.column_attrs]
#print(attr_names)

papers = []
for cp in q.all():
    print(cp.name)
    for p in cp.has_part:
        r = {k:p.__dict__[k] for k in p.__dict__ if k[0]!="_"}
        papers.append(r)
        #if p.abstract is None or len(p.abstract) == 0:
        #    continue
        #print('\t~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        #print('\t'+p.type_str)
        #rint('\t'+p.id)
        #rint('\t'+p.doi)
        #print('\t'+p.title)
        #print('\t'+p.abstract)
df_pub = pd.DataFrame(papers)
df_pub['id'] = pd.to_numeric(df_pub['id'], errors='coerce')

Nature Methods Papers/Imaging Queries
BioRxiv/Imaging Queries


In [10]:
loc1 = '/Users/gburns/Documents/2023H2/image_tech_landscaping/alhazen_analysis/p2_08-25-23/'
df1 = pd.read_csv(loc1+'df_0-200_70b.tsv', sep='\t', index_col=0)
df2 = pd.read_csv(loc1+'df_200-700_70b.tsv', sep='\t', index_col=0)
df = pd.concat([df1, df2])
df['ID_PAPER'] = pd.to_numeric(df['ID_PAPER'], errors='coerce')

In [11]:
cols_to_keep = [k for k in df_pub.columns]
cols_to_keep.append('is_imaging_method')
df_final = pd.merge(df, df_pub, left_on='ID_PAPER', right_on='id', how='left')
df_final.drop(columns=[k for k in df_final.columns if k not in cols_to_keep], inplace=True)
df_final

,is_imaging_method,id,full_text,license,format,name,type_str,doi,title,abstract,publication_date,rights,creation_date,iri
0,True,27749836.0,None,None,None,None,ScientificPrimaryResearchArticle,10.1038/nmeth.4033,Random-access scanning microscopy for 3D imagi...,Understanding how neural circuits process info...,2016-10-17,None,None,None
1,True,27749837.0,None,None,None,None,ScientificPrimaryResearchArticle,10.1038/nmeth.4031,ATAC-see reveals the accessible genome by tran...,Spatial organization of the genome plays a cen...,2016-10-17,None,None,None
2,True,27776112.0,None,None,None,None,ScientificPrimaryResearchArticle,10.1038/nmeth.4034,Bright photoactivatable fluorophores for singl...,Small-molecule fluorophores are important tool...,2016-10-24,None,None,None
3,True,27798596.0,None,None,None,None,ScientificPrimaryResearchArticle,10.1038/nmeth.4030,An acquisition and analysis pipeline for scann...,,2016-10-01,None,None,None
4,True,27798609.0,None,None,None,None,ScientificPrimaryResearchArticle,10.1038/nmeth.4046,Simultaneous dual-color fluorescence lifetime ...,We describe a red-shifted fluorescence resonan...,2016-10-31,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,True,19801991.0,None,None,None,None,ScientificPrimaryResearchArticle,10.1038/nmeth.1380,"High-resolution, long-term characterization of...","We present a single-cell motility assay, which...",2009-10-04,None,None,None
696,False,19838168.0,None,None,None,None,ScientificPrimaryResearchArticle,10.1038/nmeth.1393,A chemical platform for improved induction of ...,The slow kinetics and low efficiency of reprog...,2009-10-18,None,None,None
697,True,19838170.0,None,None,None,None,ScientificPrimaryResearchArticle,10.1038/nmeth.1390,Visual proteomics of the human pathogen Leptos...,Systems biology conceptualizes biological syst...,2009-10-18,None,None,None
698,True,19881510.0,None,None,None,None,ScientificPrimaryResearchArticle,10.1038/nmeth.1395,High-speed nanoscopic tracking of the position...,"Optical studies have revealed that, after bind...",2009-11-01,None,None,None


In [12]:
from functools import partial

from alhazen.core import TaskInstructionRegistry, get_langchain_llm, get_cached_gguf, get_langchain_embeddings, GGUF_LOOKUP_URL, MODEL_TYPE

from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain.llms import LlamaCpp 
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.schema.runnable import RunnableLambda



In [13]:
os.environ['LLMS_TEMP_DIR'] = '/Users/gburns/alhazen/'
n_gpu_layers = 1 #kwargs.get('n_gpu_layers', 1)
temperature = 0.1 #kwargs.get('temperature', 0.1)
n_batch = 512 #kwargs.get('n_batch', 512)  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
model_path = get_cached_gguf('llama-2-70b-chat')
n_ctx = 4096 #kwargs.get('n_ctx', 4096)

llm = LlamaCpp(
    model_path=model_path,
    n_ctx=n_ctx,
    n_gpu_layers=n_gpu_layers,
    temperature=temperature,
    n_batch=n_batch,
    f16_kv=True,
    verbose=True, # Verbose is required to pass to the callback manager
)    

#llm = get_langchain_llm('llama-2-70b-chat')   


llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from /Users/gburns/alhazen/llama-2-70b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  8192

In [155]:
from alhazen.utils.langchain_utils import suppress_stdout_stderr, JsonEnclosedByTextOutputParser
from langchain.schema import OutputParserException

instructions = TaskInstructionRegistry()
instructions.load_prompts_from_yaml('tiab_prompts.yaml')
pt1 = instructions.get_instruction_template('methods papers binary classifier').generate_llama2_prompt_template()

is_imaging_lcel = pt1 | llm | JsonEnclosedByTextOutputParser()

extractions = []
for i, row in df_pub.iterrows():
    print('\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print(i, row.id, row.title)
    s1 = {'title':row.title, 'abstract':row.abstract, 'id':row.id}
    
    # track how long it takes to run the extraction
    start = datetime.datetime.now()
    try:
        #with suppress_stdout_stderr():
        out2 = is_imaging_lcel.invoke(s1, config={'callbacks': [ConsoleCallbackHandler()]})
    except OutputParserException as e:
        continue
    if out2 is not None:
        out2['id'] = row.id
        out2['title'] = row.title
        extractions.append(out2)
        end = datetime.datetime.now()
        out2['time'] = end-start
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')





~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0 15782150.0 Sulforhodamine 101 as a specific marker of astroglia in the neocortex in vivo.
[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "title": "Sulforhodamine 101 as a specific marker of astroglia in the neocortex in vivo.",
  "abstract": "Glial cells have been identified as key signaling components in the brain; however, methods to investigate their structure and function in vivo have been lacking. Here, we describe a new, highly selective approach for labeling astrocytes in intact rodent neocortex that allows in vivo imaging using two-photon microscopy. The red fluorescent dye sulforhodamine 101 (SR101) was specifically taken up by protoplasmic astrocytes after brief exposure to the brain surface. Specificity was confirmed by immunohistochemistry. In addition, SR101 labeled enhanced green fluorescent protein (EGFP)-expressing astrocytes but not microglial cells in transgenic mice. We used SR101 labeling to q

Llama.generate: prefix-match hit

llama_print_timings:        load time = 75403.36 ms
llama_print_timings:      sample time =   181.09 ms /   256 runs   (    0.71 ms per token,  1413.64 tokens per second)
llama_print_timings: prompt eval time = 12778.79 ms /   563 tokens (   22.70 ms per token,    44.06 tokens per second)
llama_print_timings:        eval time = 49748.29 ms /   255 runs   (  195.09 ms per token,     5.13 tokens per second)
llama_print_timings:       total time = 63067.37 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:RunnableSequence > 3:llm:LlamaCpp] [63.07s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "  {\n\"is_method_paper\": true\n}\n\nThe title and abstract of the paper have been split into separate sentences as follows:\n\nTitle:\n\n* Sulforhodamine 101 as a specific marker of astroglia in the neocortex in vivo.\n\nAbstract:\n\n* Glial cells have been identified as key signaling components in the brain; however, methods to investigate their structure and function in vivo have been lacking.\n* Here, we describe a new, highly selective approach for labeling astrocytes in intact rodent neocortex that allows in vivo imaging using two-photon microscopy.\n* The red fluorescent dye sulforhodamine 101 (SR101) was specifically taken up by protoplasmic astrocytes after brief exposure to the brain surface.\n* Specificity was confirmed by immunohistochemistry.\n* In addition, SR101 labeled enhanced green fluorescent protein (EGFP)-expressing astrocy


llama_print_timings:        load time = 75403.36 ms
llama_print_timings:      sample time =    97.70 ms /   138 runs   (    0.71 ms per token,  1412.53 tokens per second)
llama_print_timings: prompt eval time = 10966.52 ms /   500 tokens (   21.93 ms per token,    45.59 tokens per second)
llama_print_timings:        eval time = 28619.32 ms /   137 runs   (  208.90 ms per token,     4.79 tokens per second)
llama_print_timings:       total time = 39871.87 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time = 75403.36 ms
llama_print_timings:      sample time =    92.46 ms /   131 runs   (    0.71 ms per token,  1416.84 tokens per second)
llama_print_timings: prompt eval time = 13263.76 ms /   591 tokens (   22.44 ms per token,    44.56 tokens per second)
llama_print_timings:        eval time = 28425.22 ms /   130 runs   (  218.66 ms per token,     4.57 tokens per second)
llama_print_timings:       total time = 41962.51 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:RunnableSequence > 3:llm:LlamaCpp] [41.96s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "  {\n\"is_method_paper\": true\n}\n\nThe title and abstract of the paper suggest that the main goal of the work is to develop a new imaging technique, LAMP, for measuring molecular transfer rates across gap junction connexin channels in intact living cells. The methods used by the scientists involve the use of a new class of photo-activatable fluorophores and local activation of a molecular fluorescent probe to optically label a cell. Therefore, the work is primarily concerned with developing new technology or methods, and not investigating how biological systems work.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 4:parser:JsonEnclosedByTextOutputParser] Entering Parser run with input:
{
  "input": "  {\n\"is_method_paper\": true

...


llama_print_timings:        load time = 75403.36 ms
llama_print_timings:      sample time =   156.85 ms /   115 runs   (    1.36 ms per token,   733.20 tokens per second)
llama_print_timings: prompt eval time = 12555.97 ms /   506 tokens (   24.81 ms per token,    40.30 tokens per second)
llama_print_timings:        eval time = 26053.18 ms /   114 runs   (  228.54 ms per token,     4.38 tokens per second)
llama_print_timings:       total time = 39174.87 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time = 75403.36 ms
llama_print_timings:      sample time =   296.12 ms /   178 runs   (    1.66 ms per token,   601.10 tokens per second)
llama_print_timings: prompt eval time =  9687.73 ms /   387 tokens (   25.03 ms per token,    39.95 tokens per second)
llama_print_timings:        eval time = 39372.87 ms /   177 runs   (  222.45 ms per token,     4.50 tokens per second)
llama_print_timings:       total time = 50184.31 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:RunnableSequence > 3:llm:LlamaCpp] [50.19s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "  {\n\"is_method_paper\": true\n}\n\nThe title and abstract of the paper suggest that the main goal of the work is to develop a new method for imaging molecular positions and anisotropies with nanometer-scale resolution. The methods used by the scientists, such as fluorescence photoactivation localization microscopy (FPALM) and its modification to polarization (P)-FPALM, are described in detail in the abstract. Additionally, the paper presents results obtained using these methods on mouse fibroblasts expressing Dendra2-actin or Dendra2-hemagglutinin. Therefore, it can be concluded that the work is primarily concerned with developing new technology and methods for imaging biological structures, making it a true value for the variable is_method_paper.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output


llama_print_timings:        load time = 75403.36 ms
llama_print_timings:      sample time =   236.57 ms /   146 runs   (    1.62 ms per token,   617.15 tokens per second)
llama_print_timings: prompt eval time = 12909.71 ms /   537 tokens (   24.04 ms per token,    41.60 tokens per second)
llama_print_timings:        eval time = 32044.35 ms /   145 runs   (  221.00 ms per token,     4.52 tokens per second)
llama_print_timings:       total time = 45855.50 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:RunnableSequence > 3:llm:LlamaCpp] [45.86s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "  {\n\"is_method_paper\": true\n}\n\nThe title and abstract of the paper suggest that the main goal of the work is to develop a new method for imaging cellular structures with nanometer-scale resolution using multicolor three-dimensional (3D) stochastic optical reconstruction microscopy (STORM). The methods used by the scientists, such as generating photoswitchable probes and performing 3D localization in conjunction with focal plane scanning, are described in detail in the abstract. Therefore, it can be concluded that the work is concerned with developing new technology or methods, and the variable is_method_paper should be set to true.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 4:parser:JsonEnclosedByTextOutputParser] Enteri

KeyboardInterrupt: 

In [157]:
len(extractions)

133

In [14]:
from alhazen.utils.langchain_utils import suppress_stdout_stderr, JsonEnclosedByTextOutputParser
from langchain.schema import OutputParserException

instructions = TaskInstructionRegistry()
instructions.load_prompts_from_yaml('tiab_prompts.yaml')
pt2 = instructions.get_instruction_template('technology_type method extraction 2').generate_llama2_prompt_template()

extract_lcel = pt2 | llm | JsonEnclosedByTextOutputParser()

extractions = []
for i, row in df_final.iterrows():
    if row.is_imaging_method:
        if i<138:
            continue
        print('\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print(i, row.id, row.title)
        s1 = {'title':row.title, 'abstract':row.abstract, 'id':row.id,  'is_imaging_method':row.is_imaging_method }
        
        # track how long it takes to run the extraction
        start = datetime.datetime.now()
        try:
            with suppress_stdout_stderr():
                #out2 = extract_lcel.invoke(s1, config={'callbacks': [ConsoleCallbackHandler()]})
                out2 = extract_lcel.invoke(s1)
        except OutputParserException as e:
            continue
        if out2 is not None:
            out2['id'] = row.id
            out2['doi'] = row.doi
            out2['is_imaging_method'] = row.is_imaging_method
            out2['title'] = row.title
            extractions.append(out2)
            end = datetime.datetime.now()
            out2['time'] = end-start
        print(out2)
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')




~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
138 30202059.0 Three-photon imaging of mouse brain structure and function through the intact skull.



llama_print_timings:        load time = 10656.15 ms
llama_print_timings:      sample time =    99.04 ms /    69 runs   (    1.44 ms per token,   696.67 tokens per second)
llama_print_timings: prompt eval time = 15598.63 ms /   759 tokens (   20.55 ms per token,    48.66 tokens per second)
llama_print_timings:        eval time = 12046.76 ms /    68 runs   (  177.16 ms per token,     5.64 tokens per second)
llama_print_timings:       total time = 27962.02 ms


{'text': 'Three-photon excitation provided improved optical sectioning compared with that obtained with two-photon excitation, even when we used the same excitation wavelength and imaging system.', 'name': 'none', 'technology_type': 'acquisition hardware', 'id': 30202059.0, 'doi': '10.1038/s41592-018-0115-y', 'is_imaging_method': True, 'title': 'Three-photon imaging of mouse brain structure and function through the intact skull.', 'time': datetime.timedelta(seconds=27, microseconds=967306)}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
139 30224671.0 Template-free 2D particle fusion in localization microscopy.



llama_print_timings:        load time = 10656.15 ms
llama_print_timings:      sample time =    68.33 ms /    68 runs   (    1.00 ms per token,   995.19 tokens per second)
llama_print_timings: prompt eval time = 13133.08 ms /   646 tokens (   20.33 ms per token,    49.19 tokens per second)
llama_print_timings:        eval time = 12115.92 ms /    67 runs   (  180.83 ms per token,     5.53 tokens per second)
llama_print_timings:       total time = 25442.96 ms


{'text': 'We present a template-free particle-fusion approach based on an all-to-all registration that provides robustness against individual misregistrations and underlabeling.', 'name': 'template-free particle-fusion', 'technology_type': 'data analysis', 'id': 30224671.0, 'doi': '10.1038/s41592-018-0136-6', 'is_imaging_method': True, 'title': 'Template-free 2D particle fusion in localization microscopy.', 'time': datetime.timedelta(seconds=25, microseconds=448449)}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
140 30224672.0 Label-free prediction of three-dimensional fluorescence images from transmitted-light microscopy.



llama_print_timings:        load time = 10656.15 ms
llama_print_timings:      sample time =    35.16 ms /    50 runs   (    0.70 ms per token,  1422.03 tokens per second)
llama_print_timings: prompt eval time = 13474.49 ms /   631 tokens (   21.35 ms per token,    46.83 tokens per second)
llama_print_timings:        eval time =  9058.46 ms /    49 runs   (  184.87 ms per token,     5.41 tokens per second)
llama_print_timings:       total time = 22632.99 ms


{'text': 'Label-free prediction of three-dimensional fluorescence images from transmitted-light microscopy.', 'name': 'none', 'technology_type': 'data analysis', 'id': 30224672.0, 'doi': '10.1038/s41592-018-0111-2', 'is_imaging_method': True, 'title': 'Label-free prediction of three-dimensional fluorescence images from transmitted-light microscopy.', 'time': datetime.timedelta(seconds=22, microseconds=636325)}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
141 30250056.0 Reducing effects of particle adsorption to the air-water interface in cryo-EM.
{'text': 'Rapid plunge-freezing robot and nanowire grids reduce effects of particle adsorption to air-water interface in cryo-EM.', 'name': 'none', 'technology_type': 'acquisition hardware', 'id': 30250056.0, 'doi': '10.1038/s41592-018-0139-3', 'is_imaging_method': True, 'title': 'Reducing effects of particle adsorption to the air-water interface in cryo-EM.', 'time': datetime.timedelta(seconds=25, micros

In [161]:
pd.DataFrame(extractions).to_csv('/Users/gburns/Documents/2023H2/image_tech_landscaping/alhazen_analysis/p3_23-11-03/df_1-138.tsv', sep='\t', index=False)